In [47]:
import sagemaker
from sagemaker.session import TrainingInput
import numpy as np
from sagemaker.serializers import CSVSerializer, NumpySerializer
from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()

bucket = 'datascience-sagemaker-fernandosousa'
prefix = 'perceptron/op_logicos'

instance_type = 'local' # trocar para um instância, por exemplo ml.m5.large, para treinar e implantar no sagemaker

role = sagemaker.get_execution_role()

In [48]:
# criar dados de treinamento
dados = [
      [0, 0, 0],
      [0, 1, 0],
      [1, 0, 0],
      [1, 1, 1]
    ]
x = np.array(dados)
# save to csv file
np.savetxt('data/train.csv', x, delimiter=',')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

# definir caminhos para sagemaker copiar os dados
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [49]:
estimator = PyTorch(entry_point='perceptron.py',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'error':0.1
                    })

In [50]:
estimator.fit({"training": train_input, "validation": validation_input})

Creating obzs71bi49-algo-1-78lds ... 
Creating obzs71bi49-algo-1-78lds ... done
Attaching to obzs71bi49-algo-1-78lds
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,602 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,604 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,616 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,619 sagemaker_pytorch_container.training INFO     Invoking user training script.
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,812 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,827 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
obzs71bi49-algo-1-78lds | 2022-04-21 22:27:03,841 sagemaker-training-toolkit 

In [51]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type, serializer=CSVSerializer())

4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:27:06,087 [INFO ] pool-1-thread-12 ACCESS_LOG - /172.18.0.1:53380 "GET /ping HTTP/1.1" 200 1
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:27:06,087 [INFO ] pool-1-thread-12 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:65c49bebe574,timestamp:null
!

In [54]:
response = predictor.predict(np.array([1,1,0]))
response

4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:40:17,619 [INFO ] W-9001-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:0.97|#ModelName:model,Level:Model|#hostname:65c49bebe574,requestID:92baf1f7-d636-4123-95f8-d49495c966ae,timestamp:1650580817
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:40:17,620 [INFO ] W-9001-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 2
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:40:17,620 [INFO ] W-9001-model_1 ACCESS_LOG - /172.18.0.1:53384 "POST /invocations HTTP/1.1" 200 3
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:40:17,620 [INFO ] W-9001-model_1 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:65c49bebe574,timestamp:null
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:40:17,620 [INFO ] W-9001-model_1 TS_METRICS - QueueTime.ms:0|#Level:Host|#hostname:65c49bebe574,timestamp:null
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:40:17,620 [INFO ] W-9001-model_1 TS_METRICS - WorkerThreadTime.ms:1|#Level:Host|#hostname:65c49bebe574,timestamp:null


array([0.])

4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:41:05,442 [INFO ] pool-2-thread-1 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:65c49bebe574,timestamp:1650580865
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:41:05,443 [INFO ] pool-2-thread-1 TS_METRICS - DiskAvailable.Gigabytes:7.929302215576172|#Level:Host|#hostname:65c49bebe574,timestamp:1650580865
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:41:05,443 [INFO ] pool-2-thread-1 TS_METRICS - DiskUsage.Gigabytes:100.18470001220703|#Level:Host|#hostname:65c49bebe574,timestamp:1650580865
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:41:05,443 [INFO ] pool-2-thread-1 TS_METRICS - DiskUtilization.Percent:92.7|#Level:Host|#hostname:65c49bebe574,timestamp:1650580865
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:41:05,443 [INFO ] pool-2-thread-1 TS_METRICS - MemoryAvailable.Megabytes:2310.109375|#Level:Host|#hostname:65c49bebe574,timestamp:1650580865
4t7xsdpn1l-algo-1-v8g9i | 2022-04-21 22:41:05,443 [INFO ] pool-2-thread-1 TS_METRICS - MemoryUsed.Megabytes:13